In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# ۱. آماده‌سازی دیتاست (تولید متن)
text = "علی قوی است"
tokens = text.split()
vocab = sorted(list(set(tokens)))
word_to_id = {word: i + 1 for i, word in enumerate(vocab)}
id_to_word = {i: word for word, i in word_to_id.items()}
vocab_size = len(word_to_id) + 1

# ورودی: [علی، قوی] -> خروجی هدف: [قوی، است]
x_train = np.array([[word_to_id["علی"], word_to_id["قوی"]]])
y_train = np.array([[word_to_id["قوی"], word_to_id["است"]]])

In [ ]:
# ۲. تابع ساخت ماسک مثلثی (برای جلوگیری از دیدن آینده)
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # خروجی یک ماتریس است که بالای قطر اصلی آن ۱ است

In [ ]:
# ۳. لایه Positional Encoding
class PositionalEncoding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super().__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
    def call(self, x):
        maxlen = tf.shape(x)[1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        return x + self.pos_emb(positions)

In [ ]:
# ۴. ساختار اصلی Decoder Block
class DecoderBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, training):
        # ایجاد ماسک به صورت داینامیک بر اساس طول ورودی
        mask = create_look_ahead_mask(tf.shape(x)[1])

        # Masked Multi-Head Attention
        attn_output = self.mha(x, x, x, attention_mask=mask)
        out1 = self.layernorm1(x + attn_output)

        # Feed Forward
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
# ۵. اسمبل کردن مدل نهایی
def build_decoder_model(vocab_size, embed_dim, num_heads, ff_dim):
    inputs = layers.Input(shape=(None,)) # طول ورودی متغیر
    x = layers.Embedding(vocab_size, embed_dim)(inputs)
    x = PositionalEncoding(maxlen=10, embed_dim=embed_dim)(x)

    # استفاده از ۲ لایه دکودر
    x = DecoderBlock(embed_dim, num_heads, ff_dim)(x, training=True)
    x = DecoderBlock(embed_dim, num_heads, ff_dim)(x, training=True)

    outputs = layers.Dense(vocab_size, activation="softmax")(x)
    return models.Model(inputs, outputs)

model = build_decoder_model(vocab_size, embed_dim=32, num_heads=2, ff_dim=64)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

In [ ]:
# ۶. آموزش (۱۰۰ اپوک)
model.fit(x_train, y_train, epochs=200)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 2.6572e-04
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 2.6531e-04
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 2.6477e-04
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 2.6435e-04
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 2.6388e-04
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 2.6340e-04
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 2.6292e-04
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 2.6251e-04
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 2.6197e-04
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 2.6161e-04
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 2.6108e-04
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 2.6066e-04
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 2.6030e-04
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 2.5977e-04
Epoch 15/200
1/1 ━━━━━━

In [ ]:
# ۷. تست مدل: تولید متن (Inference)
def generate_text(start_word):
    current_word = start_word
    result = [current_word]
    for _ in range(2):
        x = np.array([[word_to_id[current_word]]])
        preds = model.predict(x, verbose=0)
        next_id = np.argmax(preds[0, -1, :]) # آخرین پیش‌بینی را بردار
        current_word = id_to_word[next_id]
        result.append(current_word)
    return " ".join(result)

print("پیش‌بینی مدل برای شروع با 'علی':")
print(generate_text("علی"))

پیش‌بینی مدل برای شروع با 'علی':
علی قوی است
